Following Free Code Camps tutorial closely.

In [1]:
import ollama
import numpy as np
import torch

In [2]:
question = "What is a Kalman filter"
document = "Chapter 5: AR and Kalman Filters"

In [3]:
import tiktoken

def num_tokens_from_str(string: str, encoding_name: str) -> int:
    """Returns number of tokens in a text string"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return encoding.encode(string), num_tokens

In [4]:
num_tokens_from_str(question, "cl100k_base")

([3923, 374, 264, 27930, 1543, 4141], 6)

In [5]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = OllamaEmbeddings(model="llama3.2")

query_result = embeddings.embed_query(question)
document_result = embeddings.embed_query(document)

In [6]:
print(str(query_result)[:100])
print(len(query_result))

[-0.0073224027, -0.019477012, 0.017059486, -0.0046530142, 0.0021997404, -0.009726169, 0.008544929, -
3072


In [7]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product/(norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print(similarity)

0.6177236244940539


In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Lecture 1 - Basic circuit theory.pdf")
loader2 = PyPDFLoader("Lecture 7- The MOS transistors.pdf")
docs = loader.load()
docs2 = loader2.load()

print(f"Total characters: {len(docs[2].page_content)}")
print(f"Total characters: {len(docs2[5].page_content)}")
print(len(docs2))
print(docs2[5].page_content[:500])

Total characters: 514
Total characters: 659
42
6
 Purely symmetric device: source 
and drain are interchangeable 
depending on the applied voltages;
 To differentiate between source 
and drain, use the following 
definitions:
　 For NMOS, the source terminal is 
biased at a lower potential than 
the drain;
　 For PMOS, the source terminal is 
biased at a higher potential than 
the drain terminal.
The MOS Terminals
S
G
D
p-type substrate
B
n+ n+
 The BODY (also referred to as Bulk) terminal is usually connected to a 
fixed potential:
　 For N


In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

all_docs = docs + docs2

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300,
    chunk_overlap=50
)

#Make splits
splits = text_splitter.split_documents(all_docs)

In [11]:
vectorstore = InMemoryVectorStore.from_documents(splits, embedding=embeddings)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

From RAG Tutorial this appears to end the embedding and indexing section, next video we will see retrieval

# Part3 - Retrieval

In this section code camp goes over the retrieval section

In [12]:
docsearch = retriever.get_relevant_documents("How does an NMOS Transistor work")

/var/folders/xc/68d5fbnj61b_bgvbkv5gj8t40000gn/T/ipykernel_80254/3037671115.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docsearch = retriever.get_relevant_documents("How does an NMOS Transistor work")


In [13]:
docsearch2 = retriever.get_relevant_documents("List the basic axioms of circuit theory")

In [30]:
print(docsearch2)

[Document(id='adaf80e3-d555-4ebd-961f-95703746a7fa', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1 - Basic circuit theory.pdf', 'total_pages': 36, 'page': 0, 'page_label': '1'}, page_content='Lecture 1\nBasic circuit theory'), Document(id='7aeba4a7-9109-4459-9a05-471271c06d7a', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1 - Basic circuit theory.pdf', 'total_pages': 36, 'page': 19, 'page_label': '20'}, page_content='20\nLinear circuit e

In [31]:
print(docsearch)

[Document(id='805e7137-9b0c-48f2-aaa4-cbd20efbbaff', metadata={'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': "D:20240405072534Z00'00'", 'author': 'Wen', 'moddate': "D:20250521165548Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 7- The MOS transistors [Compatibility Mode]', 'source': 'Lecture 7- The MOS transistors.pdf', 'total_pages': 42, 'page': 2, 'page_label': '3'}, page_content='3\n50nm transistor dimension\n1.2nm Gate \nOxide\nSD\n~2000x smaller than diameter of human hair\nIntel 50nm transistor in production'), Document(id='adaf80e3-d555-4ebd-961f-95703746a7fa', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1

In [16]:
docsearch3 = retriever.get_relevant_documents("What is a linear dependent source")

print(docsearch3)

[Document(id='7aeba4a7-9109-4459-9a05-471271c06d7a', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1 - Basic circuit theory.pdf', 'total_pages': 36, 'page': 19, 'page_label': '20'}, page_content='20\nLinear circuit elements and dependent sources'), Document(id='8638bb1a-be81-4c3f-ad78-992f6dd4c9db', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1 - Basic circuit theory.pdf', 'total_pages': 36, 'page': 11, 'page_label': '12'}, page_content='

In [17]:
print(docsearch3[0].page_content)

20
Linear circuit elements and dependent sources


# Part 4 - Generation

In [24]:
from langchain.prompts import PromptTemplate

template = """Answer the following question only using the following context:
{context}

If the answer is not contained in the context, respond with:
"I cannot answer this question because the necessary information was not found in the provided documents."

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the following question only using the following context:\n{context}\n\nIf the answer is not contained in the context, respond with:\n"I cannot answer this question because the necessary information was not found in the provided documents."\n\nQuestion: {question}\n')

In [25]:
llm = OllamaLLM(model = "llama3.2")

In [26]:
chain = prompt | llm

In [27]:
print(chain.invoke({"context":all_docs, "question":"Explain the operation principle"}))

Based on the provided documents, I can explain the operation principle of a MOSFET (Metal-Oxide-Semiconductor Field-Effect Transistor).

**Basic Operation Principle**

A MOSFET is a type of field-effect transistor that uses a voltage applied to a control electrode (the gate) to create a flow of current between two other electrodes (the source and drain). The operation principle can be summarized as follows:

1. **Cut-off region**: When the gate-to-source voltage (Vgs) is negative or zero, the channel between the source and drain is completely closed, and no current flows.
2. **Linear region**: When Vgs is positive, but not high enough to fully open the channel, the MOSFET operates in the linear region, where the current through the channel increases linearly with Vgs.
3. **Saturation region**: When Vgs is sufficiently positive, the channel is fully opened, and the MOSFET enters the saturation region, where the current through the channel reaches its maximum value.

**Switching Operatio

In [28]:
print(chain.invoke({"context":all_docs, "question":"Explain what happens when there is negative gate voltage"}))

According to the document, when there is a negative gate voltage, it means that the gate-to-source potential difference is zero. This results in a channel not existing between the source and drain, as described on page 40 of the lecture notes.

In other words, with a negative gate voltage, the MOSFET is in the "cut-off" or "OFF" region, meaning current cannot pass between the source and drain.


In [29]:
print(chain.invoke({"context":all_docs, "question":"How does a bandstop filter work"}))

I couldn't find any information about a bandstop filter in the provided documents. The documents appear to be related to MOSFET transistors and their applications as switches and amplifiers, but they don't mention bandstop filters.

If you're looking for information on how a bandstop filter works, I can provide some general information:

A bandstop filter is a type of filter that allows all frequencies except those within a specific frequency range to pass through. It works by using a combination of inductive and capacitive components to create a resonant circuit that rejects the desired frequencies.

The basic principle of a bandstop filter is to use a capacitor and an inductor to create a resonant circuit at a specific frequency. When an alternating current (AC) signal is applied to the circuit, it causes the capacitor and inductor to oscillate at their natural resonance frequency. This creates a peak or null in the circuit's impedance, which effectively blocks the desired frequencie